In [ ]:

import folium

# Create a Map instance
m = folium.Map(location=[60.25, 24.8], tiles='Stamen Toner',
                   zoom_start=10, control_scale=True)

In [ ]:
 import geopandas as gpd

In [ ]:
outfp = "/Users/satwi/geopandas/base_map.html"

In [ ]:
 # Let's change the basemap style to 'Stamen Toner'
m = folium.Map(location=[40.730610, -73.935242], tiles='Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True)

 # Filepath to the output
outfp = "/Users/satwi/geopandas/base_map.html"

# Save the map
m.save(outfp)

In [ ]:
# Filepaths
fp = "/Users/satwi/geopandas/data/Vaestotietoruudukko_2015.shp"
addr_fp = "/Users/satwi/geopandas/data/addresses.shp"

# Read Data
data = gpd.read_file(fp)
ad = gpd.read_file(addr_fp)

# Re-project to WGS84
data['geometry'] = data['geometry'].to_crs(epsg=4326)
ad['geometry'] = ad['geometry'].to_crs(epsg=4326)

# Update the CRS of the GeoDataFrame
data.crs = from_epsg(4326)
ad.crs = from_epsg(4326)

# Make a selection (only data above 0 and below 1000)
data = data.ix[(data['ASUKKAITA'] > 0) & (data['ASUKKAITA'] <= 1000)]

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
data['geoid'] = data.index.astype(str)
ad['geoid'] = ad.index.astype(str)

# Select data
data = data[['geoid', 'ASUKKAITA', 'geometry']]

# Save the file as geojson
jsontxt = data.to_json()

In [ ]:
# Create a Clustered map where points are clustered
marker_cluster = folium.MarkerCluster().add_to(map_osm)


# Create Choropleth map where the colors are coming from a column "ASUKKAITA".
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
# with threshold_scale we can adjust the class intervals for the values
map_osm.choropleth(geo_str=jsontxt, data=data, columns=['geoid', 'ASUKKAITA'], key_on="feature.id",
                   fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.2, line_color='white', line_weight=0,
                   threshold_scale=[100, 250, 500, 1000, 2000],
                   legend_name='Population in Helsinki', highlight=False, smooth_factor=1.0)


# Create Address points on top of the map
for idx, row in ad.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get address information
    address = row['address']
    # Add marker to the map
    folium.RegularPolygonMarker(location=[lat, lon], popup=address, fill_color='#2b8cbe', number_of_sides=6, radius=8).add_to(marker_cluster)

# Save the output
outfp = r'/Users/satwi/geopandas/data/pop15.html'
map_osm.save(outfp)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Filepaths
grid_fp = r"/Users/satwi/geopandas/data/TravelTimes_to_5975375_RailwayStation.shp"
roads_fp = r"/Users/satwi/geopandas/data/roads.shp"
metro_fp = r"/Users/satwi/geopandas/data/metro.shp"

# Read files
grid = gpd.read_file(grid_fp)
roads = gpd.read_file(roads_fp)
metro = gpd.read_file(metro_fp)

In [ ]:
# Get the CRS of the grid
gridCRS = grid.crs

# Reproject geometries using the crs of travel time grid
roads['geometry'] = roads['geometry'].to_crs(crs=gridCRS)
metro['geometry'] = metro['geometry'].to_crs(crs=gridCRS)

In [ ]:
# Visualize the travel times into 9 classes using "Quantiles" classification scheme
# Add also a little bit of transparency with `alpha` parameter
# (ranges from 0 to 1 where 0 is fully transparent and 1 has no transparency)
my_map = grid.plot(column="car_r_t", linewidth=0.03, cmap="Reds", scheme="quantiles", k=9, alpha=0.9)

# Add roads on top of the grid
# (use ax parameter to define the map on top of which the second items are plotted)
roads.plot(ax=my_map, color="grey", linewidth=1.5)

# Add metro on top of the previous map
metro.plot(ax=my_map, color="red", linewidth=2.5)

# Remove the empty white-space around the axes
plt.tight_layout()

# Save the figure as png file with resolution of 300 dpi
outfp = r"/data/static_map.png"
plt.savefig(outfp, dpi=300)